In [ ]:
# اخر شغل
import pandas as pd
import numpy as np

# تحديد أسماء الأعمدة
columns = [
    "رقم البوستر", "عدد البيفوتات", "الشركة", "رقم البيفوت",
    "101 مساحة البيفوت", "70 مساحة البيفوت",
    "موقف الجسات تم", "موقف الجسات متبقى",
    "الصلاحية A", "الصلاحية B", "الصلاحية C", "الصلاحية D", "الصلاحية محول تنقيط",
    "المساحة المنزرعة للبيفوت", "مساحه غير الصالح", "مساحة المحول تنقيط", "موقف التسويات", "قواعد البيفوت",
    "عدد أجهزة الري المحوري", "تركيب أجهزة الري المحوري", "استلام الإشراف الزراعي"
]

phase1_path = r'جميع مراحل البيفوهات\المرحله الاولي.xlsx'
phase1_sheet_names = ['zone1', 'zone2', 'zone3']

phase2_path = r'جميع مراحل البيفوهات\00البيفوتات المرحلة الثانية.xlsx'
phase2_sheet_names = ['zone3', 'zone4', 'zone5', 'zone6', 'zone7', 'zone8', 'zone9']

phase3_path = r'جميع مراحل البيفوهات\بيفوتات المرحلة الثالثة .xlsx'
phase3_sheet_names = ['ZONE 7 (BR7)', 'ZONE 9 (BR9)', 'ZONE 9 (BR9-1)', 'ZONE 9 (BR10)', 'ZONE 10 (BR11)', 
                      'المرحلة الثالثة (zone6-zone7)',
                      'ZONE 4 (BR3)', 'ZONE 4 (BR4)', 'ZONE 5 (BR5)', 'ZONE 5 (BR5-1)']

excel_sheet_path = [phase1_path, phase2_path, phase3_path]
each_path_for_each_phase = [phase1_sheet_names, phase2_sheet_names, phase3_sheet_names]

# قائمة لتجميع البيانات
all_data = []

for path, sheet_names in zip(excel_sheet_path, each_path_for_each_phase):
    for sheet_name in sheet_names:
    # قراءة البيانات من الشيت
        df = pd.read_excel(path, sheet_name=sheet_name, 
                        names=columns, index_col=None)
        # حذف أول صفين
        df = df.iloc[2:]

        # حذف صف الإجمالي إن وجد
        if df.iloc[-1]['رقم البوستر'] == 'الاجمالى':
            df = df.iloc[:-1]

        # ffill عموديًا للأعمدة المحددة
        cols_to_fill = ['رقم البوستر', 'عدد البيفوتات', 'الشركة']
        df[cols_to_fill] = df[cols_to_fill].replace(['', '–', ' '], np.nan)
        df[cols_to_fill] = df[cols_to_fill].ffill()

        # ffill أفقيًا لأعمدة الصلاحية
        cols_to_fill_right = ["الصلاحية A", "الصلاحية B", "الصلاحية C", "الصلاحية D"]
        df[cols_to_fill_right] = df[cols_to_fill_right].apply(lambda row: row.ffill(), axis=1)

        # إضافة اسم الزون للتمييز (اختياري)
        df["الزون"] = sheet_name

        # إضافة الداتا للفائمة
        all_data.append(df)

# دمج كل البيانات في DataFrame واحد
merged_df = pd.concat(all_data, ignore_index=True)

merged_df = merged_df[~ merged_df["رقم البوستر"].isna()]
def round_number(x):
    try:
        return round(float(x))
    except:
        return x
merged_df['عدد أجهزة الري المحوري'] = merged_df['عدد أجهزة الري المحوري'].apply(round_number)

# حفظ البيانات في ملف Excel
merged_df.to_excel("اختبار دمج_البيفوتات.xlsx", index=False)

C:\Users\HP\AppData\Local\Temp\ipykernel_11784\4209878635.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols_to_fill] = df[cols_to_fill].replace(['', '–', ' '], np.nan)
C:\Users\HP\AppData\Local\Temp\ipykernel_11784\4209878635.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols_to_fill_right] = df[cols_to_fill_right].apply(lambda row: row.ffill(), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_11784\4209878635.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To